Import libraries

In [1]:
import os
import sys
import glob

import matplotlib.pyplot as plt
import cv2
import numpy as np
import importlib
from process_frames import Processor

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    import pandas as pd
    from tqdm import tqdm
    tqdm.pandas()

List images

In [2]:
path = os.getcwd()
os.chdir(path)
sys.path.append(path)
sys.path.append(os.path.join(path, 'scripts'))

path_color = '../data/color/'
path_depth = '../data/depth/'
path_mosaic = '../data/mosaic/'

#Create mosaic path if it not exits
if (not os.path.isdir(path_mosaic)):
    os.mkdir(path_mosaic)

file_list = glob.glob(path_depth+"*")
file_list = [os.path.basename(filename) for filename in file_list]

print(len(file_list),"images found")

2069 images found


Process images

In [3]:
processor = Processor()
areas = {filename: processor.process_image(depth_filename=os.path.join(path_depth, filename),
                          color_filename=os.path.join(path_color, filename),
                          mosaic_path=path_mosaic) for filename in file_list}

Prepare data

In [4]:
for area in areas:    
    if areas[area] is None:
        areas[area] = ['','','','','','','','','','','','','','']    

In [5]:
areas_dataframe = pd.DataFrame.from_dict(areas, orient='index',
     columns=['area', 'x', 'y', 'width', 'height', 'MA', 'ma', 'centroid', 'orientation', '% area', 'center distance', 'excentricity', 'perimeter', 'curvature_coef'])
areas_dataframe.index.name = 'uuid'
areas_dataframe.index = areas_dataframe.index.str.replace('.npy', '',)
areas_dataframe.head()

,area,x,y,width,height,MA,ma,centroid,orientation,% area,center distance,excentricity,perimeter,curvature_coef
uuid,,,,,,,,,,,,,,
0018b76693a14e20bce3027ff67dc055,273025,196,119,576,346,304.147,563.124,"(258.3778804886101, 474.8950997500354)",-1.34823,50.0156,54.1115,1.64309,1727.47,0.733157
001a83383a7e43e994dbf5b1796f97cc,,,,,,,,,,,,,,
00607e166c6e4054ab75451aacfbe32d,265084,186,118,592,339,307.1,574.335,"(257.15060499913113, 476.7588417673474)",-1.40705,51.4239,55.4765,1.37831,1757.66,0.741016
00635ae09a834ebf86ee85c58c12ff82,201017,109,108,565,245,250.569,523.747,"(235.90266531935276, 407.5007414600858)",-1.55165,49.9597,17.0004,0.510114,1552.63,0.666081
0066778f47734ca7a71be790f562c551,271063,195,120,578,344,306.357,563.429,"(259.0343416370107, 475.18341449709686)",-1.36629,50.4843,54.6081,1.57173,1733.13,0.736683


Read csv file with weights

In [6]:
weight_data = pd.read_csv(os.path.join("..", 'data', 'id_weight_sex_uuid.csv'), usecols=['uuid', 'id', 'sex', 'weight'])
weight_data.head()

,uuid,id,sex,weight
0,3b144fdfdb9f4e02b0f65aed674fbd72,0268,1,24.5
1,960fd92c91f541f6b85889426c39a0fe,0261,1,27.1
2,ca10f050724d47f5971c3530ef341a8d,0236,0,25.5
3,3c4dcc3c6d83425dab0d0acd75227f99,0236,0,25.5
4,46d3f67bbf684992ac945ce7c1991ae7,0238,1,25.8


Save calculated information in csv file

In [7]:
mix = pd.merge(weight_data, areas_dataframe, on='uuid', how='left')
mix.to_csv(os.path.join("..", 'data', 'id_weight_sex_uuid_area.csv'))
mix.head()

,uuid,id,sex,weight,area,x,y,width,height,MA,ma,centroid,orientation,% area,center distance,excentricity,perimeter,curvature_coef
0,3b144fdfdb9f4e02b0f65aed674fbd72,0268,1,24.5,292877,103,105,564,312,226.871,589.829,"(291.26669756191296, 390.79475760418666)",-1.3058,46.2195,61.0808,0.838312,1797.36,0.928812
1,960fd92c91f541f6b85889426c39a0fe,0261,1,27.1,308998,80,202,648,256,217.186,681.105,"(328.9446619950536, 427.0004590421944)",1.44142,58.5333,88.9953,1.37597,1741.82,0.969556
2,ca10f050724d47f5971c3530ef341a8d,0236,0,25.5,292175,195,103,569,324,167.003,671.221,"(270.91533371876403, 489.13916618487826)",1.18351,36.0346,72.1032,0.864987,1568.81,0.853786
3,3c4dcc3c6d83425dab0d0acd75227f99,0236,0,25.5,,,,,,,,,,,,,,
4,46d3f67bbf684992ac945ce7c1991ae7,0238,1,25.8,274669,117,181,613,210,188.983,637.33,"(288.28581487044687, 444.6955618663162)",1.52045,60.0394,52.534,0.751995,1589.51,0.97712
